## Advanced Operations

In [1]:
import gtn
import nb_utils
nb_utils.init()

### Intersection

The intersection of two acceptors is the set of strings which is accepted by both of them. The score of a path in the intersected graph is the sum of the score of the path in each of the input graphs. More formally the language of the intersected graph is given by $\{ x \mid x \in \mathcal{L}(A_1) \;\land\; x \in \mathcal{L}(A_2)\}$.

The analagous operation for transducers is the composition, which we will discuss in more detail in the next section. In most machine learning applications, intersection and compose tend to be the primary operations used to compute more complex graphs from simpler input graphs. Thus gaining a deeper understanding of these two operations is worth the time.

In [2]:
symbols = {0: 'a', 1: 'b', 2: 'c'}

g1 = gtn.Graph()
g1.add_node(start=True)
g1.add_node(start=False, accept=True)
g1.add_arc(src_node=0, dst_node=0, label=0)
g1.add_arc(src_node=0, dst_node=1, label=1)

g2 = gtn.Graph()
g2.add_node(start=True)
g2.add_node()
g2.add_node(start=False, accept=True)
g2.add_arc(src_node=0, dst_node=1, label=0)
g2.add_arc(src_node=0, dst_node=1, label=1)
g2.add_arc(src_node=0, dst_node=1, label=2)
g2.add_arc(src_node=1, dst_node=2, label=0)
g2.add_arc(src_node=1, dst_node=2, label=1)
g2.add_arc(src_node=1, dst_node=2, label=2)

gtn.draw(g1, "figures/intersect_1.svg", isymbols=symbols)
gtn.draw(g2, "figures/intersect_2.svg", isymbols=symbols)

We'll use the two graphs below to illustrate a general algorithm for computing the intersection. In this case, the intersected graph is easy to see by inspecting the two graphs. The only string which is recognized by both graphs is the string $ab$, hence the intersected graph is the graph which recognizes $ab$.

<div class="figure">
  <div class="img">
    <img src="figures/intersect_1.svg"/>
  </div>
  <div class="img" style="padding-top:23px;">
    <img src="figures/intersect_2.svg"/> 
  </div>
</div>

The general algortihm relies on a queue, pseudocode is given below:

```
def intersect(g1, g2):
    queue = Queue()
    
    # Add all pairs of start states to the queue
    add_start_states(queue, g1, g2)
    
    while not empty(queue):
        (u1, u2) = pop(queue)
        for arc1, arc2 in arc_pairs((u1, u2)):
          if labels_match(arc1, arc2):
              n1 = dst_node(arc1)
              n2 = dst_node(arc2)
              push(queue, (n1, n2))
              add_state(new_graph, (n1, n2))
              add_arc(new_graph, src_node=(u1, u2), dst_node=(v1, v2), label=label(arc1))
```



<table>
  <tr>
      <td><img src="figures/intersect_steps/intersect_step_1_g1.svg"/></td>
      <td><img src="figures/intersect_steps/intersect_step_1_g2.svg"/></td>
      <td><img src="figures/intersect_steps/intersect_step_1_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_2_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_2_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_2_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_3_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_3_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_3_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_4_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_4_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_4_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_5_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_5_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_5_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_5_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_5_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_5_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_6_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_6_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_6_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_7_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_7_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_7_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_8_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_8_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_8_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_9_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_9_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_9_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
  <tr>
    <td><img src="figures/intersect_steps/intersect_step_10_g1.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_10_g2.svg"/></td>
    <td><img src="figures/intersect_steps/intersect_step_10_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">This is a very long caption it is a very long caption $\epsilon$ is a caption</div></td>
  </tr>
</table>